# Template Pipeline
Normalisation&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**-**  
Imputation 1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Median**  
Outlier Detection&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Z Score**  
Imputation 2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Median**  
Feature Selection&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Implicit to model**  
Model&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Lasso Regression**


## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install xgboost


     |████████████████████████████████| 148.9 MB 142 kB/s  eta 0:00:01 15.5 MB 12.2 MB/s eta 0:00:11[K     |████████████▉                   | 59.6 MB 12.2 MB/s eta 0:00:08 eta 0:00:01��███████████▎| 145.3 MB 69.1 MB/s eta 0:00:01
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
# General
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# ML
from sklearn.linear_model import LinearRegression, Lasso, BayesianRidge, LassoCV, LassoLarsCV
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

import xgboost as xgb

# Custom
import sys,os
# %cd /content/gdrive/My Drive/ETHZ/Autumn2020/AML/Git/AML/Task_1/Solution
# sys.path.append('/content/gdrive/My Drive/ETHZ/Autumn2020/AML/Git/AML/Task_1/Solution')
sys.path.append('/home/jovyan/work/AML/Task_1/Solution') # I would like a cleaner solution but works for now
import Components.Imputation as Imputation
import Components.Outlier_Detection_1D as Outlier_Detection_1D
import Components.Outlier_Detection_ND as Outlier_Detection_ND
import Components.Feature_Selection as Feature_Selection
import Components.Normalisation as Normalisation
import Components.data_fetching as data_fetching

# CAREFUL:
# If you make changes to a custom module, you have to reload it, i.e rerun this cell
import importlib
importlib.reload(Imputation)
importlib.reload(Outlier_Detection_1D)
importlib.reload(Outlier_Detection_ND)
importlib.reload(Feature_Selection)
importlib.reload(Normalisation)
importlib.reload(data_fetching)

<module 'Components.data_fetching' from '/home/jovyan/work/AML/Task_1/Solution/Components/data_fetching.py'>

## Data Cleaning

In [ ]:
X, y = data_fetching.get_train_data()
x_test_final = data_fetching.get_test_data()

mv_mask = Imputation.missing_values_mask(X)
X = Imputation.median(X)
X = Normalisation.gaussian(X)
X,y,mv_mask = Outlier_Detection_ND.magic_indices(X,y,n_outliers=10,mask=mv_mask)
X = Outlier_Detection_1D.z_score(X) #,x_extra=x_test_final)
X = X.mask(mv_mask) # Re-impute the ones we imputed before
X = Imputation.mean(X)
# X,x_test_final = Imputation.iterative_regression2(X,x_test_final)
x_test_final = data_fetching.get_test_data() # I don't want to impute x_test_final based on X as well... data leak maybe?
x_test_final = Imputation.mean(x_test_final)
X,x_test_final = Feature_Selection.select_percentile_mut_inf(X,y,x_test=x_test_final,percent=50)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

/opt/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass percentile=50 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/opt/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


## Model Training

### Test

In [ ]:
gbm = xgb.XGBRegressor(max_depth=3, n_estimators=75, learning_rate=0.05, alpha=1, objective='reg:squarederror').fit(x_train, y_train)

kfold = KFold(n_splits=10)
results = cross_val_score(gbm, x_train, y_train, cv=kfold)
print(results)
print(results.mean())

print('Train Score')
y_pred_train = gbm.predict(x_train)
test_score = r2_score(y_train, y_pred_train)
print(test_score)

print('Test Score')
y_pred_test = gbm.predict(x_test)
test_score = r2_score(y_test, y_pred_test)
print(test_score)

[0.52196672 0.52751118 0.52042321 0.44937897 0.45876363 0.58604072
 0.52967402 0.49833196 0.6046143  0.51960716]
0.521631187746819
Train Score
0.7553048082770955
Test Score
0.5988531138208393


## Final Prediction

In [ ]:
gbm = xgb.XGBRegressor(max_depth=3, n_estimators=75, learning_rate=0.05, objective='reg:squarederror').fit(X, y)

y_pred = gbm.predict(x_test_final)
plt.hist(y_pred)
print("Train Score:", r2_score(y, gbm.predict(X)))

y_pred_pd = pd.DataFrame(data=y_pred, columns=["y"])
# y_pred_pd.to_csv('../../Predictions/RFE40_GradientBoosting.csv', index_label='id')

In [ ]:
# Random test
X2, y2 = data_fetching.get_train_data()
x_test_final2 = data_fetching.get_test_data()

gbm = xgb.XGBRegressor(objective='reg:squarederror').fit(X2, y2)

y_pred = gbm.predict(x_test_final2)
plt.hist(y_pred)
print("Train Score:", r2_score(y2, gbm.predict(X2)))

# y_pred_pd = pd.DataFrame(data=y_pred, columns=["y"])
# y_pred_pd.to_csv('../../Predictions/RFE40_GradientBoosting.csv', index_label='id')